In [ ]:
%load_ext autoreload
%autoreload 2

import dask.dataframe as dd
import os
import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt
%matplotlib inline

DEVICE = 'cuda'

# 1. Import Dataset

### 1.1 Loading raw data

In [ ]:
data_folder = str(os.getcwd()) + "\data"

blumenstock_dynamic_types = {'LOAN_SEQUENCE_NUMBER': str, 'MONTHLY_REPORTING_PERIOD': str,'CURRENT_ACTUAL_UPB': float, 'CURRENT_LOAN_DELINQUENCY_STATUS': float, 
                    'CURRENT_INTEREST_RATE':float,'ELTV': float ,'LOAN_AGE': float, 'REMAINING_MONTHS_TO_LEGAL_MATURITY': float, 'CREDIT_SCORE': float,
                    'DTI': float, 'LTV': float, 'BAL_REPAID': float, 
                    'LABEL': float, "TIME_TO_EVENT": float, 'ORIGINAL_INTEREST_RATE': float, 'ORIGINAL_UPB': float, 'TOTAL_OBSERVED_LENGTH': float}

df_blumenstock_dynamic = dd.read_parquet(data_folder + "/blumenstock_dynamic_labeled_sample_orig_*.parquet.gzip")
df_blumenstock_dynamic = df_blumenstock_dynamic.astype(blumenstock_dynamic_types)

In [ ]:
df_blumenstock_dynamic.head(10)

### 1.2 Normalising raw data

In [ ]:
covariates_to_normalise = ['CURRENT_ACTUAL_UPB', 'CURRENT_LOAN_DELINQUENCY_STATUS', 'CURRENT_INTEREST_RATE', 'ELTV', 'LOAN_AGE', 'REMAINING_MONTHS_TO_LEGAL_MATURITY', 'CREDIT_SCORE',
                            'DTI', 'LTV', 'BAL_REPAID', 'ORIGINAL_INTEREST_RATE', 'ORIGINAL_UPB']

df_blumenstoch_dynamic_mean = df_blumenstock_dynamic[covariates_to_normalise].mean().compute()
df_blumenstoch_std = df_blumenstock_dynamic[covariates_to_normalise].std().compute()

df_blumenstock_dynamic[covariates_to_normalise] = (df_blumenstock_dynamic[covariates_to_normalise] - df_blumenstoch_dynamic_mean) / df_blumenstoch_std


In [ ]:
df_blumenstock_dynamic.head(10)

In [ ]:
df_blumenstock_dynamic.columns

### 1.3 Creating dataloaders

In [ ]:
from torch.utils.data import DataLoader
from FREDDIEMAC_main_data import FREDDIEMAC_main_dataset, FREDDIEMAC_main_dataloader

BATCH_SIZE = 2**8

frac_cases=0.25
test_set = False
augment = False
data_augment_factor = 3
random_state = 42

allowed_covariates = ['CURRENT_ACTUAL_UPB', 'CURRENT_LOAN_DELINQUENCY_STATUS', 'CURRENT_INTEREST_RATE', 'ELTV', 
                             'LOAN_AGE', 'REMAINING_MONTHS_TO_LEGAL_MATURITY', 'CREDIT_SCORE', 'DTI', 'LTV', 'BAL_REPAID', 
                             'ORIGINAL_INTEREST_RATE', 'ORIGINAL_UPB']


print("number of covariates = ", len(allowed_covariates))

TIME_TO_EVENT_covariate = 'TIME_TO_EVENT'
TOTAL_OBSERVED_LENGTH_covariate = 'TOTAL_OBSERVED_LENGTH'
LABEL_covariate = 'LABEL'


FREDDIEMAC_raw_dataset = FREDDIEMAC_main_dataset(df_blumenstock_dynamic, 
                                                    allowed_covariates,
                                                    TIME_TO_EVENT_covariate,
                                                    TOTAL_OBSERVED_LENGTH_covariate,
                                                    LABEL_covariate,
                                                    frac_cases,
                                                    random_state,
                                                    test_set,
                                                    augment,
                                                    data_augment_factor)

print("This dataset will contain %d samples" % len(FREDDIEMAC_raw_dataset))

data_loader = FREDDIEMAC_main_dataloader(dataset=FREDDIEMAC_raw_dataset,
                                    batch_size=BATCH_SIZE)

print("This dataloader will deliver %d batches" % data_loader.get_max_iterations())

batch_data, batch_data_length, batch_event, batch_tte = next(data_loader)

print(batch_data.shape)
print(batch_data_length.shape)
print(batch_event.shape)
print(batch_tte.shape)

### Data Exploration

In [ ]:
batch_data, batch_data_length, batch_event, batch_tte = next(iter(data_loader))

for i in range(min(BATCH_SIZE, 16)):
    print("batch event= %d --- batch_data_length= %d --- batch_tte= %d" % (batch_event[i], batch_data_length[i], batch_tte[i]))

# 2. Hyperparameters

# 3. Defining The Model